<a href="https://colab.research.google.com/github/SumanthS117/1BM23CS348-AiLab/blob/main/firstorderlogic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Define rules and facts in a simple form
print("sumanth S Shetty 1bm23cs348")
# Facts (initial knowledge base)
facts = {
    "American(Robert)",
    "Enemy(CountryA, America)",
    "Missile(M1)",
    "Has(CountryA, M1)",
    "Sold(Robert, CountryA, M1)"
}

# Rules (implications)
rules = [
    # If a country is enemy to America, then it is hostile
    ("Enemy(X, America)", "Hostile(X)"),

    # If Robert sold weapons to a hostile nation and Robert is American, Robert is a criminal
    ("American(Robert) & Sold(Robert, Y, Z) & Hostile(Y) & Weapon(Z)", "Criminal(Robert)"),

    # Define what is a weapon (missile is a weapon)
    ("Missile(Z)", "Weapon(Z)")
]

# Parse facts and rules simply using sets and strings
def forward_chaining(facts, rules, goal):
    new_facts = set(facts)
    while True:
        added = False
        for premise, conclusion in rules:
            # Try to match premise to facts (handle & conjunctions)
            premises = premise.split(" & ")
            substitutions = [{}]  # List of possible substitutions

            for p in premises:
                new_substitutions = []
                for sub in substitutions:
                    # Substitute variables in p using sub
                    p_inst = substitute(p, sub)
                    matches = match_fact(p_inst, new_facts)
                    for m in matches:
                        # Merge substitutions
                        merged = merge_substitutions(sub, m)
                        if merged is not None:
                            new_substitutions.append(merged)
                substitutions = new_substitutions

            # Apply substitutions to conclusion and add to facts
            for sub in substitutions:
                conclusion_inst = substitute(conclusion, sub)
                if conclusion_inst not in new_facts:
                    new_facts.add(conclusion_inst)
                    print(f"Inferred: {conclusion_inst}")
                    added = True
                    if conclusion_inst == goal:
                        print("Goal reached!")
                        return True
        if not added:
            break
    return goal in new_facts

# Substitute variables with their bindings
def substitute(expr, sub):
    for var, val in sub.items():
        expr = expr.replace(var, val)
    return expr

# Match premise to facts with variable bindings
def match_fact(premise, facts):
    # premise like Predicate(arg1,arg2,...)
    import re
    pattern = re.compile(r"(\w+)\(([^)]*)\)")
    m = pattern.match(premise)
    if not m:
        return []
    pred = m.group(1)
    args = m.group(2).split(",")
    args = [a.strip() for a in args]
    matches = []
    for f in facts:
        fm = pattern.match(f)
        if not fm:
            continue
        fpred = fm.group(1)
        fargs = fm.group(2).split(",")
        fargs = [a.strip() for a in fargs]
        if fpred != pred or len(fargs) != len(args):
            continue
        sub = {}
        failed = False
        for a, fa in zip(args, fargs):
            if is_variable(a):
                if a in sub and sub[a] != fa:
                    failed = True
                    break
                sub[a] = fa
            else:
                if a != fa:
                    failed = True
                    break
        if not failed:
            matches.append(sub)
    return matches

# Merge two substitutions if compatible
def merge_substitutions(s1, s2):
    s = s1.copy()
    for k,v in s2.items():
        if k in s and s[k] != v:
            return None
        s[k] = v
    return s

# Check if a term is a variable (single uppercase letter)
def is_variable(x):
    return len(x) == 1 and x.isupper()

# Run the forward chaining to prove Criminal(Robert)
goal = "Criminal(Robert)"
if forward_chaining(facts, rules, goal):
    print("Robert is criminal: Proven")
else:
    print("Could not prove Robert is criminal")


sumanth S Shetty 1bm23cs348
Inferred: Hostile(CountryA)
Inferred: Weapon(M1)
Inferred: Criminal(Robert)
Goal reached!
Robert is criminal: Proven
